# T4 : SMS Spam Detector

Semester 2221, CSEC 520/620, Team 4\
Assignment 1 - SMS Spam Detector\
Due Sep 15, 2022 11:59 PM EST\
Accounts for 12% of total grade.

## Description

Welcome to Team 4's SMS Spam Detector. This assignment's goal is to examine both k-NN and Naive Bayes classifiers for determining whether an SMS message is spam or not spam. We will provide some performance metrics in our analsysis to hopefully determine which is more appropriate for this type of classification.

## Requirements

- Python 3
- Must have the `SMSSpamCollection` file in the same directory as this file. 

We must import the various modules and libraries that we will depend on during execution.

In [ ]:
import copy
import itertools
import math
import os
from os.path import exists
import random
import re

## Utility Methods

Similarly to the import statements, we utilize the below utility methods across our models/notebook. This includes detagging tokens, calculating and printing metrics, etc. Run the code block in this section to ensure our utility methods are defined.

In [ ]:
def detag_tokens(tokens):
    """
    Detags a tagged tokens list. Removes the first element of each child list.

    :param tokens: Two dimensional list, with the first element of each child list being a tag ham/spam.
    :return: Detagged tokens list.
    """
    # Copy of tokens, without the ham/spam tag.
    detagged_tokens = copy.deepcopy(tokens)
    detagged_tokens = [token[1:] for token in detagged_tokens]

    return detagged_tokens

In [ ]:
def separate_tags(tokens):
    """
    Separates the list of tagged tokens based on the tags ham/spam.

    :param tokens: Array containing arrays of individual words. The first word in each array must be either "ham" or "spam".
    :return: Dictionary object containing separated "ham" and "spam" sets.
    """
    tokens = copy.deepcopy(tokens)

    separated_set = {"ham": list(filter(lambda token: token[0] == "ham", tokens)),
                     "spam": list(filter(lambda token: token[0] == "spam", tokens))}
    return separated_set

In [ ]:
def print_metrics(dictionary, is_percentage=True):
    """
    Prints metrics from a dictionary, metric name is the key and metric result is the value.

    :param dictionary: Dictionary containing metric name and metric value.
    :param is_percentage: Flag that determines if dictionary metric values will be printed as a percentage.
    :return: None
    """

    if is_percentage:
        for metric in dictionary:
            print(f'{metric + ":":>20} {dictionary[metric]:024.20%}')
    else:
        for metric in dictionary:
            print(f'{metric + ":":>20} {dictionary[metric]}')

In [ ]:
def calculate_metrics(tp, fp, tn, fn, print_results=True, print_only_percentages=True, title=None):
    """
    Calculates classification performance metrics. Can also handle printing of metrics.

    :param tp: Number of true positive predictions.
    :param fp: Number of false positive predictions.
    :param tn: Number of true negative predictions.
    :param fn: Number of false negative predictions.
    :param print_results: Flag that determines whether results will be printed. True by default.
    :param print_only_percentages: Flag that determines whether only percentages, and not raw numbers will be printed. True by default.
    :param title: Title of classifier the calculated metrics belong to. None by default.
    :return: Dictionary of calculated metrics.
    """
    # Calculate metrics
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    # Bundle metrics into dictionary
    base_metrics = {"True Positive": tp, "False Positive": fp, "True Negative": tn, "False Negative": fn}
    additional_metrics = {"Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1-Score": f1_score}
    result_metrics = {**base_metrics, **additional_metrics}

    if print_results:
        # Heading
        header = " Resulting Performance Metrics "
        print("-" * 45)
        print(header.center(45, "-"))
        if title: print(title.center(len(header), " ").center(45, "-"))
        print("-" * 45)

        # TP, FP, TN, and FN Numbers
        if not print_only_percentages:
            print_metrics(base_metrics, False)

        # Calculate TP, FP, TN, and FN Rates
        total_predictions = sum(base_metrics.values())
        rates = {}
        for metric in base_metrics:
            rates[metric + " Rate"] = base_metrics[metric] / total_predictions

        # Print All Percentages
        print_metrics(rates)
        print("-----".center(45, " "))
        print_metrics(additional_metrics)

    # Return dictionary of metrics
    return result_metrics

## Tokenizing

Our model begins by performing tokenization on the dataset. This takes every line of the file and essentially separates and sanitizes each word.

In [ ]:
def tokenize(filename, print_info=False):
    """
    Performs sanitization and then tokenization on the given file.

    :param filename: The name or path of the file that contains the data to perform tokenization on.
    :param print_info: Flag that determines whether information will be printed. False by default.
    :return: An array of sanitized tokens derived from the data housed in the given file.
    """
    file = open(filename, 'r')
    lines = [line for line in file]

    # First, convert special characters into spaces
    clean_lines = [re.sub('\W+', ' ', line) for line in lines]

    # Second, separate each word in each line while also ensuring lowercase
    tokens = [line.lower().split() for line in clean_lines]

    # Print information
    if print_info:
        print(f'{"Lines:":>18} {lines}')
        print(f'{"Cleaned Lines:":>18} {clean_lines}')
        print(f'{"Tokens:":>18} {tokens}')

    # Return sanitized tokens
    return tokens

In [ ]:
def split_dataset(og_list, percent_train=0.8):
    """
    Splits the original dataset into the training and testing sets. Testing set allocation size is derived from the given training set percentage. Token selection is performed randomly.

    :param og_list: The original set to split into training and testing sets.
    :param percent_train: The percentage, in decimal form, of the original set to allocate towards training data.
    :return: Dictionary object containing allocated "train" and "test" sets.
    """
    # Get the total number of tokens in the original list
    og_total = len(og_list)

    # Setup and determine training and testing set allocation
    percent_test = round(og_total * (1 - percent_train))
    train_set = copy.deepcopy(og_list)
    test_set = []

    # Fill up the testing set's allocated size by randomly choosing a token from the training set and moving it to the testing set
    while percent_test > 0:
        selected_token = random.choice(train_set)
        test_set.append(selected_token)
        train_set.remove(selected_token)
        percent_test -= 1

    return {"train": train_set, "test": test_set}

Execute the code block below to perform tokenization and then split the tokens into training and testing sets.

In [ ]:
# Perform tokenization on the "SMSSpamCollection" file
tagged_tokens = tokenize('SMSSpamCollection')

# Generate dictionary containing "train" and "test" sets.
dataset = split_dataset(tagged_tokens)

## k Nearest Neighbors

In General for K-NN The algorithm would probably be:

  Compute TF-IDF vector generation

  K-NN comparison stuff

  Classify

  Compare Against cannon for confusion matrix(TPR, TNR, FPR, FNR)

This next part will be my sub-par attempt to implement TF-IDF

In [ ]:
def find_unique(og_list):
    """
    Flattens and strips all duplicate words in a given array. Then returns a list of only the unique words, while printing out facts.

    :param og_list: Array, can be multidimensional, to find all unique words in.
    :return: One dimensional array of all unique words found in the given array.
    """
    # Copy of tokens, without the ham/spam tag.
    untag_tokens = copy.deepcopy(og_list)
    untag_tokens = [token[1:] for token in untag_tokens]

    # Flatten array (2D -> 1D) and remove duplicate words.
    unique_tokens = list(itertools.chain.from_iterable(untag_tokens))
    unique_tokens = [*set(unique_tokens)]

    # Calculate Totals
    untagged_total = sum([len(token) for token in untag_tokens])
    unique_total = len(unique_tokens)
    print(f'{"Total Words:":>19} {untagged_total}')
    print(f'{"Total Unique Words:":>19} {unique_total}')

    # return unique words and untagged tokens arrays
    return untag_tokens, unique_tokens

untagged_tokens, unique_words  = find_unique(dataset.get("train"))

### TF-IDF Calculations
It has become clear that we need to find a way to include all dimensions in our calculations

In [ ]:
# First, we get a list of the IDFs for all words
# wordsdone is to be sure a word doesn't have more than one entry in the list.
idfs = []
wordsdone = []

# The total document count is always the length of the tokens list
doccount=len(untagged_tokens)
for word in unique_words:
  # Determine the number of documents that have the current word in it
  docswithword = 0
  for token in untagged_tokens:
    if word in token:
      docswithword = docswithword + 1
  if word not in wordsdone:
    if docswithword == 0:
      thisidf = [word, 0]
      idfs.append(thisidf)
    else:
      # Calc the IDF (document count/# of documents with word in it) and add it to the list.
      thisidf = [word, doccount/docswithword]
      idfs.append(thisidf)
    wordsdone.append(word)
print(idfs)

In [ ]:
# Then we calculate the tf for each word, and use it to get the tf-idf for them.
# After calculating them, we write the values of each token to a collective file.
if not exists("tf_calculations.txt"):
  tf_idfs = open("tf_calculations.txt", "x")
  tf_idfs = open("tf_calculations.txt", "w")


  for i in range(len(untagged_tokens)):
    # Add an array to separate each token from each other within the list.
    tokenlist = []
    for word in unique_words:
      token = untagged_tokens[i]
      # Get the number of times word appears in this token, along with the number of words in it total
      thiswordcount = 0
      totalwords = len(token)
      for t in token:
        if word == t:
          thiswordcount = thiswordcount + 1
      if thiswordcount != 0:
        # Get the tf (number of times given word appears/total word count in token)
        tf = math.log(thiswordcount/totalwords)
        for j in range(len(idfs)):
          tok = idfs[j]
          if word == tok[0]:
            # Get the TF-IDF (tf*idf) and write it to the file.
            val = tf*tok[1]
            #print(str(val))
            tf_idfs.write(word + "," + str(val) + " ")
      else:
        #print(word)
        tf_idfs.write(word + ",0 ")
    tf_idfs.write("\n")

Testing of KNN

In [ ]:
def idf_values2(sample):
    for i in range(len(sample)):
        tf_idfs = []
        for word in unique_words:
            token = sample[i]
            #Get the number of times word appears in this token, along with the number
            #of words in it total
            thiswordcount = 0
            totalwords = len(token)
            for t in token:
              if word == t:
                thiswordcount = thiswordcount + 1
            if thiswordcount != 0:
            #Get the tf (number of times given word appears/total word count in token)
              tf = math.log(thiswordcount/totalwords)
              for j in range(len(idfs)):
                tok = idfs[j]
                if word == tok[0]:
                #Get the TF-IDF (tf*idf) and add it to the list.
                  val = tf*tok[1]
                  vallist = [word, val]
                  tf_idfs.append(vallist)
            else:
                vallist = [word, 0]
                tf_idfs.append(vallist)
    return tf_idfs

In [ ]:
def filebasedknn(unknown, k):
    valuesfile = open("tf_calculations.txt", "r")
    neighbors=[]
    #At the start of the function, a for loop of all tf_idf values are compared 
    #against the unknown
    linenum = 0
    #print("running knn")
    for line in valuesfile:
        #print("line " + str(linenum))
                
        distance=0
        pairs = line.split()
        #for every vector in training_data, a euclidean distance is 
        #calculated against unknown
        for i in range(len(pairs)):
          currentpair = pairs[i].split(",")
          if (float(currentpair[1]) == float(0)) and (float(unknown[0][-1]) == float(0)):
            continue
          #print(i)
          if currentpair[0][0] == "'":
            currentpair[0] = currentpair[0].strip("\'")
          else:
            currentpair[0] = currentpair[0].strip('\"')
          
          #print(f'{"pairs[i]:":>19} {pairs[i]}')
          #print(f'{"currentpair:":>19} {currentpair}')
          #print(f'{"i:":>19} {i}')
          #rint(f'{"currentpair[1]:":>19} {currentpair[1]}')
          #print(f'{"unknown[0][-1]:":>19} {unknown[0][-1]}')
          #if i == 270:
              #print(currentpair)


          #The below is the Euclidean distance calculations.
          d=math.pow(float(currentpair[1])-float(unknown[0][-1]), 2)          
          distance=distance+d
        distance=math.pow(distance, .5)
        
        #if any of the vectors have a distance less than or equal to k,
        #it is put in a set of neighbors
        if len(neighbors) < k:
            neighbors.append([linenum, distance])
        else:
            for pair in neighbors:
              if distance < pair[1]:
                neighbors.remove(pair)
                neighbors.append([linenum, distance])
                break
        linenum = linenum + 1

    #The identity of neighbours are checked and the identity with the most
    #neighbours is the classifier of the unknown
    #print(k)
    spamC=0
    hamC=0
    for nn in neighbors:
      indexnum = nn[0]
      entry = tagged_tokens[indexnum][0]
      if entry == "spam":
          spamC=spamC+1
      elif entry == "ham":
          hamC=hamC+1
    if hamC > spamC:
        return "ham"
    elif spamC > hamC:
        return "spam"

In [ ]:
def tknn(tSet):
    # Numbers of each to calculate rate later
    tp, fp, tn, fn = 0, 0, 0 , 0

    guess = ""

    msgCount = 0
    tcount = 0
    while tcount < 2:
        unknown = idf_values2(tSet[tcount])
        guess = filebasedknn(unknown, 5)
        msgCount = msgCount + 1

        if tSet[tcount][0] == 'ham':
            if guess == 'ham':
                tn += 1
            elif guess == 'spam':
                fp += 1
            elif tSet[tcount][0] == 'spam':
                if guess == 'ham':
                    fn += 1
                elif guess == 'spam':
                    tp += 1

        tcount = tcount + 1

    calculate_metrics(tp, fp, tn, fn, title="K-Nearest Neighbors")


tknn(dataset.get('test'))

## Naive Bayes Function

This function is used to format the idfs of the sample we are trying to identify so that it matches the formatting of the entries in the tf-idf file.

In [ ]:
def naive_bayes_training(ham, spam):
    """
    Performs preliminary naive bayes calculations on the ham and spam set.

    :param ham: Array containing the ham set.
    :param spam: Array containing the spam set.
    :return: Percentages and calculates that will be used by the naive bayes classifier.
    """
    hamWordCounts = {}
    spamWordCounts = {}
    hamTotal = 0
    spamTotal = 0

    for msg in ham:
        for word in msg:
            hamTotal += 1
            if word in hamWordCounts:
                hamWordCounts[word] = hamWordCounts[word] + 1
            else:
                hamWordCounts[word] = 1

    for msg in spam:
        for word in msg:
            spamTotal += 1
            if word in spamWordCounts:
                spamWordCounts[word] = spamWordCounts[word] + 1
            else:
                spamWordCounts[word] = 1

    addNum = 0
    for word in hamWordCounts:
        if word not in spamWordCounts:
            if addNum != 1:
                addNum = 1
                hamTotal *= 2
                spamTotal *= 2
            spamWordCounts[word] = 0

    for word in spamWordCounts:
        if word not in hamWordCounts:
            hamWordCounts[word] = 0
            if addNum != 1:
                addNum = 1
                hamTotal *= 2
                spamTotal *= 2

    hamWPerc = {}
    for key in hamWordCounts:
        hamWordCounts[key] = hamWordCounts[key] + addNum
        hamWPerc[key] = hamWordCounts[key] / hamTotal

    spamWPerc = {}
    for key in spamWordCounts:
        spamWordCounts[key] = spamWordCounts[key] + addNum
        spamWPerc[key] = spamWordCounts[key] / spamTotal

    initHam = hamTotal / (hamTotal + spamTotal)
    initSpam = spamTotal / (hamTotal + spamTotal)

    return hamWPerc, spamWPerc, initHam, initSpam


separated_tokens = separate_tags(tagged_tokens)
hamWPerc, spamWPerc, initHam, initSpam = naive_bayes_training(separated_tokens.get("ham"), separated_tokens.get("spam"))

### Naive Bayes Classifier

This function performs the classificaiton on new unseen data. Essentially an "intelligent guessing" machine using our previosuly calculated metrics.

In [ ]:
def naive_bayes_classifier(msg):
    """
    Performs naive bayes classification to determine whether a provided message is spam or ham.

    :param msg: A message to perform classification on.
    :return: The classification result "ham" or "spam".
    """
    ham_prob = math.log(initHam)
    spam_prob = math.log(initSpam)

    for word in msg:
        if word != msg[0] and (word != "ham" or word != "spam"):
            if word in hamWPerc:
                ham_prob += math.log(hamWPerc[word])

    for word in msg:
        if word != msg[0] and (word != "ham" or word != "spam"):
            if word in spamWPerc:
                spam_prob += math.log(spamWPerc[word])

    if ham_prob > spam_prob:
        return "ham"
    elif spam_prob > ham_prob:
        return "spam"

Testing phase of Naive Bayes

In [ ]:
def naive_bayes_testing(test_set):
    """
    Performs naive bayes classification on the test set, and compares the results against their actual values.

    :param test_set: Test set that the model hasn't been trained on.
    :return: None
    """
    # Values to calculate rates later
    tpn = 0
    tnn = 0
    fpn = 0
    fnn = 0

    for msg in test_set:
        guess = naive_bayes_classifier(msg)
        if msg[0] == 'ham':
            if guess == 'ham':
                tnn += 1
            elif guess == 'spam':
                fpn += 1
        elif msg[0] == 'spam':
            if guess == 'ham':
                fnn += 1
            elif guess == 'spam':
                tpn += 1

    calculate_metrics(tpn, fpn, tnn, fnn, title="Naive Bayes")

naive_bayes_testing(dataset.get('test'))